In [ ]:
!pip install langgraph langchain-core langchain typing_extensions pydantic rich google-generativeai graphviz langchain-google-genai

In [ ]:
!pip install google-genai

In [3]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
import json
from datetime import date
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, get_buffer_string, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing_extensions import Optional, Annotated, List, Sequence, Literal
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
import operator

console = Console()

In [13]:
# TOOL DEFINITION
@tool
def get_today_date() -> str:
    """Return today's date in dd/mm/yy format."""
    return date.today().strftime("%d/%m/%y")

def show_prompt(prompt_text: str, title: str = "Prompt", border_style: str = "blue"):
    """
    Display a prompt with rich formatting and XML tag highlighting.

    Args:
        prompt_text: The prompt string to display
        title: Title for the panel (default: "Prompt")
        border_style: Border color style (default: "blue")
    """
    # Create a formatted display of the prompt
    formatted_text = Text(prompt_text)
    formatted_text.highlight_regex(r'<[^>]+>', style="bold blue")  # Highlight XML tags
    formatted_text.highlight_regex(r'##[^#\n]+', style="bold magenta")  # Highlight headers
    formatted_text.highlight_regex(r'###[^#\n]+', style="bold cyan")  # Highlight sub-headers

    # Display in a panel for better presentation
    console.print(Panel(
        formatted_text,
        title=f"[bold green]{title}[/bold green]",
        border_style=border_style,
        padding=(1, 2)
    ))

def format_message_content(message):
    """Convert message content to displayable string"""
    parts = []
    tool_calls_processed = False

    # Handle main content
    if isinstance(message.content, str):
        parts.append(message.content)
    elif isinstance(message.content, list):
        # Handle complex content like tool calls (Anthropic format)
        for item in message.content:
            if item.get('type') == 'text':
                parts.append(item['text'])
            elif item.get('type') == 'tool_use':
                parts.append(f"\n Tool Call: {item['name']}")
                parts.append(f"   Args: {json.dumps(item['input'], indent=2)}")
                parts.append(f"   ID: {item.get('id', 'N/A')}")
                tool_calls_processed = True
    else:
        parts.append(str(message.content))

    # Handle tool calls attached to the message (OpenAI format) - only if not already processed
    if not tool_calls_processed and hasattr(message, 'tool_calls') and message.tool_calls:
        for tool_call in message.tool_calls:
            parts.append(f"\n Tool Call: {tool_call['name']}")
            parts.append(f"   Args: {json.dumps(tool_call['args'], indent=2)}")
            parts.append(f"   ID: {tool_call['id']}")

    return "\n".join(parts)

def format_messages(messages):
    """Format and display a list of messages with Rich formatting"""
    for m in messages:
        msg_type = m.__class__.__name__.replace('Message', '')
        content = format_message_content(m)

        if msg_type == 'Human':
            console.print(Panel(content, title="Human", border_style="blue"))
        elif msg_type == 'Ai':
            console.print(Panel(content, title="Assistant", border_style="green"))
        elif msg_type == 'Tool':
            console.print(Panel(content, title="Tool Output", border_style="yellow"))
        else:
            console.print(Panel(content, title=f" {msg_type}", border_style="white"))

In [14]:
# STATE CLASSES
class AgentInputState(MessagesState):
    """Input state for the full agent- only contain messages from the user input """
    pass

class AgentState(MessagesState):
    """Main state for the full multiagent research system.
    Extends MessagesState with additional fields for research coordination.
    Note: some fields are duplicated across different state classes for proper
    state management between subgraph and main workflow"""

    research_brief: Optional[str]
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages]
    raw_notes: Annotated[list[str], operator.add] = []
    notes: Annotated[list[str], operator.add] = []
    final_report: str

class ClarifyWithUser(BaseModel):
    """Schema for user clarification decision and questions."""

    need_clarification: bool = Field(
        description="Whether the user needs to be asked a clarifying question.",
    )
    question: str = Field(
        description="A question to ask the user to clarify the report scope",
    )
    verification: str = Field(
        description="Verify message that we will start research after the user has provided the necessary information.",
    )

class ResearchQuestion(BaseModel):
    """Schema for structured research brief generation."""

    research_brief: str = Field(
        description="A research question that will be used to guide the research.",
    )

In [15]:
# PROMPT TEMPLATES
clarify_with_user_instructions = """
These are the messages that have been exchanged so far from the user asking for the report:
<Messages>
{messages}
</Messages>

Today's date is {date}.

Assess whether you need to ask a clarifying question, or if the user has already provided enough information for you to start research.
IMPORTANT: If you can see in the messages history that you have already asked a clarifying question, you almost always do not need to ask another one. Only ask another question if ABSOLUTELY NECESSARY.

If there are acronyms, abbreviations, or unknown terms, ask the user to clarify.
If you need to ask a question, follow these guidelines:
- Be concise while gathering all necessary information
- Make sure to gather all the information needed to carry out the research task in a concise, well-structured manner.
- Use bullet points or numbered lists if appropriate for clarity. Make sure that this uses markdown formatting and will be rendered correctly if the string output is passed to a markdown renderer.
- Don't ask for unnecessary information, or information that the user has already provided. If you can see that the user has already provided the information, do not ask for it again.

Respond in valid JSON format with these exact keys:
"need_clarification": boolean,
"question": "<question to ask the user to clarify the report scope>",
"verification": "<verification message that we will start research>"

If you need to ask a clarifying question, return:
"need_clarification": true,
"question": "<your clarifying question>",
"verification": ""

If you do not need to ask a clarifying question, return:
"need_clarification": false,
"question": "",
"verification": "<acknowledgement message that you will now start research based on the provided information>"

For the verification message when no clarification is needed:
- Acknowledge that you have sufficient information to proceed
- Briefly summarize the key aspects of what you understand from their request
- Confirm that you will now begin the research process
- Keep the message concise and professional
"""

transform_messages_into_research_topic_prompt = """You will be given a set of messages that have been exchanged so far between yourself and the user.
Your job is to translate these messages into a detailed and comprehensive research brief that will be used to guide the research.

The messages that have been exchanged so far between yourself and the user are:
<Messages>
{messages}
</Messages>

Today's date is {date}.

You will return a comprehensive research brief in markdown format that includes:
- A main research question
- Key areas to investigate
- Specific aspects and dimensions to cover
- Research methodology guidance

Structure your response as a detailed research brief, not just a single question.

Guidelines:
1. Maximize Specificity and Detail
- Include all known user preferences and explicitly list key attributes or dimensions to consider.
- It is important that all details from the user are included in the instructions.

2. Handle Unstated Dimensions Carefully
- When research quality requires considering additional dimensions that the user hasn't specified, acknowledge them as open considerations rather than assumed preferences.
- Example: Instead of assuming "budget-friendly options," say "consider all price ranges unless cost constraints are specified."
- Only mention dimensions that are genuinely necessary for comprehensive research in that domain.

3. Avoid Unwarranted Assumptions
- Never invent specific user preferences, constraints, or requirements that weren't stated.
- If the user hasn't provided a particular detail, explicitly note this lack of specification.
- Guide the researcher to treat unspecified aspects as flexible rather than making assumptions.

4. Distinguish Between Research Scope and User Preferences
- Research scope: What topics/dimensions should be investigated (can be broader than user's explicit mentions)
- User preferences: Specific constraints, requirements, or preferences (must only include what user stated)
- Example: "Research coffee quality factors (including bean sourcing, roasting methods, brewing techniques) for San Francisco coffee shops, with primary focus on taste as specified by the user."

5. Use the First Person
- Phrase the request from the perspective of the user.

6. Sources
- If specific sources should be prioritized, specify them in the research question.
- For product and travel research, prefer linking directly to official or primary websites (e.g., official brand sites, manufacturer pages, or reputable e-commerce platforms like Amazon for user reviews) rather than aggregator sites or SEO-heavy blogs.
- For academic or scientific queries, prefer linking directly to the original paper or official journal publication rather than survey papers or secondary summaries.
- For people, try linking directly to their LinkedIn profile, or their personal website if they have one.
- If the query is in a specific language, prioritize sources published in that language.
"""

In [ ]:
# CONFIGURATION
  # Replace with actual API key
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

# Prompt the user to enter the API key without showing it
api_key = getpass("Enter your Google API Key: ")

# Set it as an environment variable
os.environ['GOOGLE_API_KEY'] = api_key
# Use a generally available model name
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0)

In [17]:
# HELPER FUNCTIONS
from langchain_core.messages import ToolMessage # Import ToolMessage

def get_current_date_from_messages(messages: list) -> str:
    """Extract today's date from tool call results in messages"""
    for message in reversed(messages):  # Start from most recent
        if isinstance(message, ToolMessage):
            # The content should be the date string from get_today_date tool
            return message.content

    # If no tool message found, return error message - tool must be called first
    return "Date not available - tool not called"

In [18]:
# WORKFLOW NODES
def get_date_node(state: AgentState):
    """Node that calls the get_today_date tool"""
    # Create a model with the date tool
    model_with_tools = model.bind_tools([get_today_date])

    # Call the tool to get today's date
    response = model_with_tools.invoke([
        HumanMessage(content="What is today's date? Please use the get_today_date tool.")
    ])

    # Handle tool calls in the response
    messages_to_add = [response]

    if response.tool_calls:
        for tool_call in response.tool_calls:
            if tool_call["name"] == "get_today_date":
                tool_result = get_today_date.invoke({})
                messages_to_add.append(
                    ToolMessage(content=tool_result, tool_call_id=tool_call["id"])
                )

    return {"messages": messages_to_add}

def clarify_with_user(state: AgentState) -> Command[Literal["write_research_brief", "__end__"]]:
    """
    Determine if the user's request contains sufficient information to proceed with research.

    Uses structured output to make deterministic decisions and avoid hallucination.
    Routes to either research brief generation or ends with a clarification question.
    """
    # Get current date from the messages
    current_date = get_current_date_from_messages(state["messages"])

    # Set up structured output model
    structured_output_model = model.with_structured_output(ClarifyWithUser)

    # Invoke the model with clarification instructions
    response = structured_output_model.invoke([
        HumanMessage(content=clarify_with_user_instructions.format(
            messages=get_buffer_string(messages=state["messages"]),
            date=current_date
        ))
    ])

    # Route based on clarification need
    if response.need_clarification:
        return Command(
            goto=END,
            update={"messages": [AIMessage(content=response.question)]}
        )
    else:
        return Command(
            goto="write_research_brief",
            update={"messages": [AIMessage(content=response.verification)]}
        )

def write_research_brief(state: AgentState):
    """
    Transform the conversation history into a comprehensive research brief.

    Uses structured output to ensure the brief follows the required format
    and contains all necessary details for effective research.
    """
    # Get current date from the messages
    current_date = get_current_date_from_messages(state["messages"])

    # Set up structured output model
    structured_output_model = model.with_structured_output(ResearchQuestion)

    # Generate research brief from conversation history with date included in prompt
    response = structured_output_model.invoke([
        HumanMessage(content=transform_messages_into_research_topic_prompt.format(
            messages=get_buffer_string(state.get("messages", [])),
            date=current_date
        ))
    ])

    # Ensure the date is included in the research brief if not already there
    research_brief_with_date = response.research_brief
    if f"**Research Date:** {current_date}" not in research_brief_with_date and f"Research Date: {current_date}" not in research_brief_with_date:
        # If the LLM didn't include the date, prepend it
        if research_brief_with_date.startswith("# "):
            # Insert date after the main heading
            lines = research_brief_with_date.split('\n', 2)
            if len(lines) >= 2:
                research_brief_with_date = f"{lines[0]}\n\n**Research Date:** {current_date}\n\n{lines[1] if len(lines) > 1 else ''}"
        else:
            # Prepend date at the beginning
            research_brief_with_date = f"**Research Date:** {current_date}\n\n{research_brief_with_date}"

    # Update state with generated research brief and pass it to the supervisor
    return {
        "research_brief": research_brief_with_date,
        "supervisor_messages": [HumanMessage(content=f"{research_brief_with_date}.")]
    }

In [19]:
#  GRAPH CONSTRUCTION
# Build the scoping workflow
deep_researcher_builder = StateGraph(AgentState, input_schema=AgentInputState)

# Add workflow nodes
deep_researcher_builder.add_node("get_date", get_date_node)
deep_researcher_builder.add_node("clarify_with_user", clarify_with_user)
deep_researcher_builder.add_node("write_research_brief", write_research_brief)

# Add workflow edges
deep_researcher_builder.add_edge(START, "get_date")
deep_researcher_builder.add_edge("get_date", "clarify_with_user")
deep_researcher_builder.add_edge("write_research_brief", END)

# Compile the workflow
scope_research = deep_researcher_builder.compile()

In [20]:
#  EXAMPLE USAGE
if __name__ == "__main__":
    from langgraph.checkpoint.memory import InMemorySaver

    checkpointer = InMemorySaver()
    scope = deep_researcher_builder.compile(checkpointer=checkpointer)

    # Example conversation thread
    thread = {"configurable": {"thread_id": "1"}}
    result = scope.invoke(
        {"messages": [HumanMessage(content="Ocean plastic pollution")]},
        config=thread
    )

    print("=== CONVERSATION RESULT ===")
    format_messages(result['messages'])

    if 'research_brief' in result:
        print("\n=== RESEARCH BRIEF ===")
        from rich.markdown import Markdown
        console.print(Markdown(result['research_brief']))

=== CONVERSATION RESULT ===


╭───────────────────────────────────────────────────── Human ─────────────────────────────────────────────────────╮
│ Ocean plastic pollution                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Tool Call: get_today_date                                                                                      │
│    Args: {}                                                                                                     │
│    ID: cbec5d71-3116-465e-8953-d67fa5a892c7                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ 17/09/25                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│ To provide the most relevant report on ocean plastic pollution, could you please specify what aspects you'd     │
│ like to focus on? For example, are you interested in its causes, environmental impacts, solutions, or specific  │
│ geographical areas?                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
result = scope.invoke(
        {"messages": [HumanMessage(content="environmental impacts")]},
        config=thread
    )

print("=== CONVERSATION RESULT ===")
format_messages(result['messages'])

=== CONVERSATION RESULT ===


╭───────────────────────────────────────────────────── Human ─────────────────────────────────────────────────────╮
│ Ocean plastic pollution                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Tool Call: get_today_date                                                                                      │
│    Args: {}                                                                                                     │
│    ID: cbec5d71-3116-465e-8953-d67fa5a892c7                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ 17/09/25                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│ To provide the most relevant report on ocean plastic pollution, could you please specify what aspects you'd     │
│ like to focus on? For example, are you interested in its causes, environmental impacts, solutions, or specific  │
│ geographical areas?                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Human ─────────────────────────────────────────────────────╮
│ environmental impacts                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Tool Call: get_today_date                                                                                      │
│    Args: {}                                                                                                     │
│    ID: bc60b1e5-7ca9-4add-b86d-363db7468821                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ 17/09/25                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────  AI ──────────────────────────────────────────────────────╮
│ Thank you for clarifying. I will now start researching the environmental impacts of ocean plastic pollution.    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:
from rich.markdown import Markdown
console.print(Markdown(result['research_brief']))

Research Date: 17/09/25                                                                                            

I require a comprehensive research brief focusing on the environmental impacts of ocean plastic pollution. The     
research should address the main research question: 'What are the key environmental impacts of ocean plastic       
pollution globally?'                                                                                               

Key areas to investigate include:                                                                                  

 • Impacts on marine life (e.g., ingestion, entanglement, habitat destruction, disruption of food webs).           
 • Impacts on marine ecosystems and habitats (e.g., coral reefs, mangroves, seagrass beds).                        
 • Potential indirect impacts on human health through the marine food chain.                                       
 • Impacts on coastal environments and their associated services.                                                  
 • Long-term ecological consequences of plastic accumulation in oceans.                                            

Specific aspects and dimensions to cover should include:                                                           

 • Marine Life: Detailed examination of how macroplastics, microplastics, and nanoplastics affect various marine   
   organisms, from plankton and invertebrates to fish, marine mammals, and seabirds. This includes physical harm   
   (entanglement, ingestion leading to starvation or internal injury), chemical contamination (leaching of toxins  
   from plastics, bioaccumulation), and behavioral changes.                                                        
 • Marine Ecosystems: Analysis of the degradation and alteration of critical marine habitats due to plastic        
   accumulation, such as coral reef smothering, damage to seagrass beds, and disruption of benthic communities.    
 • Human Health: Investigation into the pathways of plastic and associated contaminants into the human diet via    
   seafood consumption, and the potential health risks (e.g., endocrine disruption, carcinogenicity) associated    
   with these exposures.                                                                                           
 • Coastal Environments: Assessment of the aesthetic and ecological degradation of beaches and coastal zones,      
   impacts on coastal biodiversity, and potential economic consequences for tourism and fisheries.                 
 • Types of Plastic Pollution: Differentiation between the impacts of macroplastics (larger debris), microplastics 
   (particles <5mm), and nanoplastics (particles <100nm), considering their distinct pathways and effects.         
 • Geographical and Temporal Scope: While the primary focus is global, the research should highlight specific      
   regions or ecosystems that are particularly vulnerable or heavily impacted. The research should cover current   
   impacts and project potential long-term ecological changes.                                                     

Research methodology guidance:                                                                                     

 • The research should primarily involve a comprehensive literature review of peer-reviewed scientific articles,   
   reports from reputable environmental organizations (e.g., UNEP, IUCN, WWF), and governmental scientific bodies. 
 • Prioritize studies that offer robust empirical evidence and meta-analyses.                                      
 • Case studies of specific marine species or ecosystems severely affected by plastic pollution should be included 
   to illustrate impacts.                                                                                          
 • While the user's explicit focus is on environmental impacts, the research should acknowledge that the causes of 
   ocean plastic pollution, potential solutions, 